In [ ]:
import pandas as pd    
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Считаем файлы в Pandas датафрейм
train = pd.read_csv('../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip', delimiter="\t")
test = pd.read_csv('../input/word2vec-nlp-tutorial/testData.tsv.zip', delimiter="\t")

In [ ]:
# Посмотрим на тренировочный датафрейм
train.head()

In [ ]:
# Посмотрим на тестовый датафрейм
test.head()

In [ ]:
# Узнаем число пропущенных значений
train.isnull().sum()

In [ ]:
# Узнаем количество столбцов и строк в датафреймах
train.shape, test.shape

In [ ]:
# Посмотрим на первый отзыв в тренировочной выборке
train.review[0]

In [ ]:
# Над признаком review проведем следующие преобразования:
# убрать HTML теги, используя библиотеку BeautifulSoup 
# оставить в признаке только буквы
# перевести весь текст в нижний регистр и разбить текст на отдельные слова
# уберем из текста стоп слова (те, которые не несут за собой смысл)

def clean_text(text):
    text = BeautifulSoup(text).get_text() 
    letters_only = re.sub("[^a-zA-Z]", " ", text) 
    words = letters_only.lower().split()                           
    stops = set(stopwords.words("english"))                   
    main_words = [w for w in words if not w in stops]   
    return( " ".join( main_words )) 

In [ ]:
# Применим функцию к тренировочной выборке
train['clean_review'] = train['review'].apply(clean_text)
train.head()

In [ ]:
# Признак review у тестовой выборки тоже необходимо преобразовать
# Потому к test тоже применим функцию 
test['clean_review'] = test['review'].apply(clean_text)
test.head()

In [ ]:
# Далее нужно перейти к числовому представлению подчищенных рецензий
# так как далее для predict будем использовать лес, который работает
# с числовыми значениями.
# Для этого используем словарь слов, который считает вхождение слов в текст
# Ограничим словарь 10000 так как обработка большего количества 
# занимает слишком много времени
vectorizer = CountVectorizer(analyzer = "word", max_features = 10000) 

In [ ]:
# Построим словарь слов и преобразуем результат в массив
X_train = vectorizer.fit_transform(train.clean_review)
X_train = X_train.toarray()
y_train = train.sentiment

In [ ]:
# Узнаем размер полученной выборки
X_train.shape

In [ ]:
# Такие же изменения необходимо проделать и с review в тестовой выборке
X_test = vectorizer.transform(test.clean_review)
X_test = X_test.toarray()

In [ ]:
# Создадим случайный лес
rf = RandomForestClassifier(n_estimators = 100)
rf.fit(X_train, y_train)

In [ ]:
# Делаем предсказания
result = rf.predict(X_test)

In [ ]:
# И выведем результат
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "submission.csv", index=False)